<a href="https://colab.research.google.com/github/scruzrafael/atividadePraticaNLP/blob/main/atividadePraticaRU2094296.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MOTIVAÇÃO DO TRABALHO**

*Com a grande difusão de notícias falsas pelas redes sociais, faz-se necessário ferramentas automáticas para verificação da veracidade das mesmas.
Identificar a veracidade de uma notícia é um trabalho complexo até mesmo para profissionais
da área de jornalismo e linguística. Sendo assim, uma ferramenta automatizada seria de grande
ajuda para a sociedade.
Sabendo disso, idealizamos esta atividade para que você possa treinar e aplicar seus
conhecimentos adquiridos nas aulas teóricas e práticas, ao mesmo tempo que aplica seus
conhecimentos em um projeto real e de grande utilidade pública.*

*Utilizado o corpus FakeBr (https://github.com/roneysco/Fake.br-Corpus), que
contém 7200 instâncias com 28 atributos e 1 rótulo em cada entrada/instância.
Este corpus está rotulado de forma binária, sendo classificado em FAKE os textos falsos (3600 entradas) e REAL os textos verdadeiros (3600 entradas). Utilizado também imagens do repositório https://github.com/scruzrafael/atividadePraticaNLP.*
  
*Abaixo código referente a Atividade Prática. De já agradeço a atenção pela verificação da apresentação*


**Rafael Sousa Cruz - RU 2094296**  
*Natural Language Processing*

*Tecnologia em Ciências de Dados*

*Uninter*


In [ ]:
#instalar bibliotecas utilizadas no projeto
!pip install notebook wordcloud pillow numpy bs4 tqdm gitpython

In [ ]:
import git
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, f_classif
from wordcloud import WordCloud, STOPWORDS
import numpy as np
import PIL.Image

In [ ]:
#importar arquivo do repositorio
def clone_github_repository(owner, repo, destination_path):
    repository_url = f"https://github.com/{owner}/{repo}.git"

    try:
        git.Repo.clone_from(repository_url, destination_path)
        print(f"Repositório clonado com sucesso em {destination_path}")
    except Exception as e:
        print(f"Falha ao clonar o repositório: {e}")

# Substitua 'roneysco' e 'Fake.br-Corpus' como os valores reais do proprietário e repositório
clone_github_repository(owner='roneysco', repo='Fake.br-Corpus', destination_path='./Fake.br-Corpus')

In [ ]:
#importa arquivo csv com as informações de colunas selecionadas
# caminho'/content/Fake.br-Corpus/preprocessed/pre-processed.csv' arquivo CSV
ru_2094296 = '/content/Fake.br-Corpus/preprocessed/pre-processed.csv'

# encoding='utf-8' para lidar com caracteres acentuados
# Se houver problemas, tente utilizar encoding='latin1' ou encoding='ISO-8859-1'

df_colunas = ['label', 'preprocessed_news']
df = pd.read_csv(ru_2094296, encoding='utf-8', usecols=df_colunas, low_memory = False)

#Visualizando as 10 primeiras linhas do DataFrame
df.head(10)

In [ ]:
#importar arquivos para imagens que serão utilizadas para carregar as wordclouds fake e true
#git https://github.com/scruzrafael/atividadePratica-imagens
def clone_github_repository(owner, repo, destination_path):
    repository_url = f"https://github.com/{owner}/{repo}.git"

    try:
        git.Repo.clone_from(repository_url, destination_path)
        print(f"Repositório clonado com sucesso em {destination_path}")
    except Exception as e:
        print(f"Falha ao clonar o repositório: {e}")

# Substitua 'scruzrafael' e 'atividadePratica-imagens' como os valores reais do proprietário e repositório
clone_github_repository(owner='scruzrafael', repo='atividadePratica-imagens', destination_path='./atividadePratica-imagens')

In [ ]:
#quantidade de instâncias
len(df)

In [ ]:
# coluna chamada 'label'
contagem_rotulos = df['label'].value_counts()

# Exibindo a contagem de cada rótulo
print(contagem_rotulos)

In [ ]:
# coluna que contém as notícias pré-processadas
coluna_noticias = 'preprocessed_news'

# variável para armazenar a contagem total de palavras
total_palavras = 0

# Itere sobre as linhas do DataFrame
for indice, linha in df.iterrows():
    # string da coluna 'preprocessed_news'
    noticia = linha[coluna_noticias]

    # Dividir a string em palavras
    palavras = noticia.split()

    # contagem de palavras da linha ao total
    total_palavras += len(palavras)

# Exiba o total de palavras
print("Total de palavras no corpus:", total_palavras)

In [ ]:
# Dividindo os dados em conjuntos de treinamento e teste
df_treino, df_teste = train_test_split(df, test_size=0.25, random_state=42)

In [ ]:
# DataFrame com coluna 'preprocessed_news'
coluna_noticias = 'preprocessed_news'

# Criar um objeto TF-IDF Vectorizer com bigramas
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))

# Aplicar o TF-IDF ao conjunto de treinamento
X_tfidf = tfidf_vectorizer.fit_transform(df_treino[coluna_noticias])

# Obter as estatísticas desejadas (por exemplo, as top N palavras)
# Neste exemplo, estamos usando SelectKBest para selecionar as top 50 palavras, 10% do total mostrado nas wordclouds (fake e true)
k_best = SelectKBest(f_classif, k=50)
X_tfidf_top = k_best.fit_transform(X_tfidf, df_treino['label'])

# Exibir as top 10 palavras
feature_names = tfidf_vectorizer.get_feature_names_out()
top_feature_indices = k_best.get_support(indices=True)
top_features = [feature_names[i] for i in top_feature_indices]

print("Top 50 palavras usando TF-IDF e bigramas:")
print(top_features)

In [ ]:
# Separar as features (X) e os rótulos (y) do conjunto de treinamento
X_treino = df_treino[coluna_noticias]
y_treino = df_treino['label']

# Separar as features (X) e os rótulos (y) do conjunto de teste
X_teste = df_teste[coluna_noticias]
y_teste = df_teste['label']

modelo = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('classificador', SVC())
])

# Treinar o modelo
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
previsoes = modelo.predict(X_teste)

# Avaliar a acurácia
acuracia = accuracy_score(y_teste, previsoes)

print("Acurácia do modelo:", acuracia)

# Exibir o relatório de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_teste, previsoes))

In [ ]:
# Separar os dados rotulados como TRUE
df_real = df[df['label'] == 'true']

# Separar os dados em conjunto de treinamento e teste
df_real_treino, df_real_teste = train_test_split(df_real, test_size=0.25, random_state=42)

# Separar as features (X) e os rótulos (y) do conjunto de treinamento e teste
X_real_treino = df_real_treino[coluna_noticias]
y_real_treino = df_real_treino['label']

X_real_teste = df_real_teste[coluna_noticias]
y_real_teste = df_real_teste['label']

# Criar um modelo de classificação usando Multinomial Naive Bayes
modelo_real = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1, 3))),  # Altere o ngram_range conforme necessário
    ('classificador', MultinomialNB())
])

# Treinar o modelo
modelo_real.fit(X_real_treino, y_real_treino)

# Fazer previsões no conjunto de teste
previsoes_real = modelo_real.predict(X_real_teste)

# Avaliar a acurácia do modelo
acuracia_real = accuracy_score(y_real_teste, previsoes_real)

# Obter as features (palavras, bigramas, trigramas)
features = modelo_real.named_steps['vectorizer'].get_feature_names_out()

# Exibir a quantidade de palavras, bigramas e trigramas
quantidade_palavras = sum(' ' not in feature for feature in features)
quantidade_bigramas = sum(' ' in feature for feature in features)
quantidade_trigramas = sum(feature.count(' ') == 2 for feature in features)

print("Quantidade de palavras usadas:", quantidade_palavras)
print("Quantidade de bigramas usados:", quantidade_bigramas)
print("Quantidade de trigramas usados:", quantidade_trigramas)
print("Acurácia do modelo para textos rotulados como TRUE:", acuracia_real)

In [ ]:
# Separar os dados rotulados como FAKE
df_real = df[df['label'] == 'fake']

# Separar os dados em conjunto de treinamento e teste
df_real_treino, df_real_teste = train_test_split(df_real, test_size=0.25, random_state=42)

# Separar as features (X) e os rótulos (y) do conjunto de treinamento e teste
X_real_treino = df_real_treino[coluna_noticias]
y_real_treino = df_real_treino['label']

X_real_teste = df_real_teste[coluna_noticias]
y_real_teste = df_real_teste['label']

# Criar um modelo de classificação usando Multinomial Naive Bayes
modelo_real = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1, 3))),  # Altere o ngram_range conforme necessário
    ('classificador', MultinomialNB())
])

# Treinar o modelo
modelo_real.fit(X_real_treino, y_real_treino)

# Fazer previsões no conjunto de teste
previsoes_real = modelo_real.predict(X_real_teste)

# Avaliar a acurácia do modelo
acuracia_real = accuracy_score(y_real_teste, previsoes_real)

# Obter as features (palavras, bigramas, trigramas)
features = modelo_real.named_steps['vectorizer'].get_feature_names_out()

# Exibir a quantidade de palavras, bigramas e trigramas
quantidade_palavras = sum(' ' not in feature for feature in features)
quantidade_bigramas = sum(' ' in feature for feature in features)
quantidade_trigramas = sum(feature.count(' ') == 2 for feature in features)

print("Quantidade de palavras usadas:", quantidade_palavras)
print("Quantidade de bigramas usados:", quantidade_bigramas)
print("Quantidade de trigramas usados:", quantidade_trigramas)
print("Acurácia do modelo para textos rotulados como FAKE:", acuracia_real)

In [ ]:
#importar máscara da imagem de fundo TRUE
imagem_mascara = np.array(PIL.Image.open("/content/atividadePratica-imagens/positiv.jpg"))

imagem_mascara

In [ ]:
# Adicione as previsões ao DataFrame de teste
df_teste['previsoes'] = previsoes

# Separe os textos por classe
text_real = ' '.join(df_teste[df_teste['previsoes'] == 'true'][coluna_noticias])

wordcloud = WordCloud(background_color='white',
                      max_words=500,
                      mask=imagem_mascara,
                      contour_color="grey",
                      contour_width=0.3,
                      min_font_size=3,
                      random_state=42).generate(text_real)

In [ ]:
# Mostrando a imagem
def gerar_wordcloud(texto, titulo):
  plt.figure(figsize = (8, 8), facecolor = None)
  plt.imshow(wordcloud)

  #plt.title(titulo)

#remove os eixos do gráfico
  plt.axis("off")
  plt.tight_layout(pad=0)
  plt.text(-0.07, 0.5, titulo, transform=plt.gca().transAxes, rotation=90, va='center', ha='left', fontsize=14)
  plt.show()

# Gerar nuvem de palavras para FAKE
gerar_wordcloud(text_real, 'RU2094296 - RAFAEL SOUSA CRUZ')

In [ ]:
#gravar imagem no caminho /content/wordCloudTrue.png
wordcloud.to_file('/content/wordCloudTrue.png')

In [ ]:
#importar máscara da imagem de fundo FAKE
imagem_mascara = np.array(PIL.Image.open("/content/atividadePratica-imagens/negativ.jpg"))

imagem_mascara

In [ ]:
# Adicione as previsões ao DataFrame de teste
df_teste['previsoes'] = previsoes

# Separe os textos por classe
text_fake = ' '.join(df_teste[df_teste['previsoes'] == 'fake'][coluna_noticias])

wordcloud = WordCloud(background_color='white',
                      max_words=500,
                      mask=imagem_mascara,
                      contour_color="grey",
                      contour_width=0.3,
                      min_font_size=3,
                      random_state=42).generate(text_fake)

In [ ]:
# Mostrando a imagem
def gerar_wordcloud(texto, titulo):
  plt.figure(figsize = (8, 8), facecolor = None)
  plt.imshow(wordcloud)

  #plt.title(titulo)

#remove os eixos do gráfico
  plt.axis("off")
  plt.tight_layout(pad=0)
  plt.text(-0.07, 0.5, titulo, transform=plt.gca().transAxes, rotation=90, va='center', ha='left', fontsize=14)
  plt.show()

# Gerar nuvem de palavras para FAKE
gerar_wordcloud(text_fake, 'RU2094296 - RAFAEL SOUSA CRUZ')

In [ ]:
#gravar imagem no caminho /content/wordCloudFake.png
wordcloud.to_file('/content/wordCloudFake.png')